# todo
- [x] code no bound
- [x] code grid output
- [x] add css
- [x] save numpy matrices
- [x] put all in python
- [x] run on meso
- [ ] code plots in notebook

In [1]:
# imports
import sys, os
import numpy as np
import glob
import datetime
import json
from pathlib import Path
from IPython.display import clear_output

# MRI analysis imports
from prfpy.stimulus import PRFStimulus2D
from prfpy.model import Iso2DGaussianModel, CSS_Iso2DGaussianModel
from prfpy.fit import Iso2DGaussianFitter, CSS_Iso2DGaussianFitter
import nibabel as nb
import cortex

sys.path.append("{}/../../../../utils".format(os.getcwd()))
from surface_utils import make_surface_image , load_surface

In [2]:
# data filenames and folder
subject = 'sub-03'
# dir_data = '/home/mszinte/disks/meso_S/data/RetinoMaps'
# dir_code = '/home/mszinte/disks/meso_H/projects/RetinoMaps'

dir_data = '/Users/uriel/disks/meso_shared/RetinoMaps'
dir_code = '/Users/uriel/disks/meso_H/projects/RetinoMaps'

pycortex_dir = "{}/derivatives/pp_data/cortex".format(dir_data)
input_vd = '{}/derivatives/vdm/vdm.npy'.format(dir_data)
input_fn_fsnative = '{}/derivatives/pp_data/{}/func/fmriprep_dct_avg/fsnative/{}_task-pRF_hemi-L_fmriprep_dct_avg_bold.func.gii'.format(
    dir_data, subject, subject)

# prf_fit_test_dir = "{}/derivatives/pp_data/{}/prf/fit/test".format(
#     dir_data, subject)
# os.makedirs(prf_fit_test_dir, exist_ok=True)

In [4]:
# pycortex config file
pycortex_db = "{}/db/".format(pycortex_dir)
pycortex_cm = "{}/colormaps/".format(pycortex_dir)
pycortex_config_file  = cortex.options.usercfg
pycortex_config_file_new = pycortex_config_file[:-4] + '_new.cfg'
pycortex_config_file_old = pycortex_config_file[:-4] + '_old.cfg'

Path(pycortex_config_file_new).touch()
with open(pycortex_config_file, 'r') as fileIn:
    with open(pycortex_config_file_new, 'w') as fileOut:
        for line in fileIn:
            if 'filestore' in line:
                newline = 'filestore=' + pycortex_db
                fileOut.write(newline)
                newline = '\n'
            elif 'colormaps' in line:
                newline = 'colormaps=' + pycortex_cm
                fileOut.write(newline)
                newline = '\n'
            else:
                newline = line
            fileOut.write(newline)
os.rename(pycortex_config_file,pycortex_config_file_old)
os.rename(pycortex_config_file_new, pycortex_config_file)

In [3]:
# analysis parameters
with open('{}/analysis_code/settings.json'.format(dir_code)) as f:
    json_s = f.read()
    analysis_info = json.loads(json_s)
screen_size_cm = analysis_info['screen_size_cm']
screen_distance_cm = analysis_info['screen_distance_cm']
TR = analysis_info['TR']
max_ecc_size = analysis_info['max_ecc_size']
gauss_grid_nr = analysis_info['gauss_grid_nr']
css_grid_nr = analysis_info['css_grid_nr']

n_jobs = 32
n_batches = 32
rsq_iterative_th = 0.01
verbose = False
grid_verbose, iterative_verbose = False, False
rois = ['occ', 'par', 'frt']

In [4]:
# load visual design
vdm = np.load(input_vd)

# determine visual design
stimulus = PRFStimulus2D(screen_size_cm=screen_size_cm[1], 
                         screen_distance_cm=screen_distance_cm,
                         design_matrix=vdm, 
                         TR=TR)

In [7]:
# Load fsnative data 
data_img_fsnative, data_fsnative = load_surface(input_fn_fsnative)

In [7]:
# Get testing vertices
roi_verts = cortex.get_roi_verts(subject=subject, 
                                 roi=rois, 
                                 mask=True)

vert_mask_occ = roi_verts['occ']
vert_mask_occ = vert_mask_occ[0:data_fsnative.shape[1]]
data_occ = data_fsnative[:, vert_mask_occ].T

vert_mask_par = roi_verts['par']
vert_mask_par = vert_mask_par[0:data_fsnative.shape[1]]
data_par = data_fsnative[:, vert_mask_par].T

vert_mask_frt = roi_verts['frt']
vert_mask_frt = vert_mask_frt[0:data_fsnative.shape[1]]
data_frt = data_fsnative[:, vert_mask_frt].T

data = np.concatenate((data_occ,data_par,data_frt))
data = data[0:10,:]

ValueError: could not convert string to float: ''

### Gaussian model

In [10]:
# define gauss model
gauss_model = Iso2DGaussianModel(stimulus=stimulus)

# grid fit gauss model
gauss_fitter = Iso2DGaussianFitter(data=data, 
                                   model=gauss_model, 
                                   n_jobs=n_jobs)

# fit parameters
gauss_params_num = 8
gauss_iterative_bound = True
gauss_iterative_repeat = 15

In [11]:
# gaussian grid fit
# -----------------

# define saving data matrices
gauss_fit_mat_repeat = np.zeros((data.shape[0], gauss_params_num, gauss_iterative_repeat+2))
gauss_pred_mat_repeat = np.zeros((data.shape[0], data.shape[1], gauss_iterative_repeat+2))
gauss_duration_repeat = np.zeros(gauss_iterative_repeat+2)

# defind grid values
ecc_gauss_grid = max_ecc_size * np.linspace(0.25,1.0,gauss_grid_nr)**2
sizes_guass_grid = max_ecc_size * np.linspace(0.1,1.0,gauss_grid_nr)**2
polars_gauss_grid = np.linspace(0.0, 2.0*np.pi, gauss_grid_nr)

# run grid fit
start_time = datetime.datetime.now()
gauss_fitter.grid_fit(ecc_grid=ecc_gauss_grid, 
                      size_grid=sizes_guass_grid, 
                      polar_grid=polars_gauss_grid, 
                      verbose=grid_verbose,
                      n_batches=n_batches);

clear_output(wait=True)

end_time = datetime.datetime.now()
duration = end_time - start_time
print("Grid fit:\t\t\t{}\nEnd time:\t\t\t{}\nDuration:\t\t\t{}".format(
    start_time, end_time, duration))

# arrange data
gauss_fit = gauss_fitter.gridsearch_params
gauss_fit_mat = np.zeros((data.shape[0],gauss_params_num))
gauss_pred_mat = np.zeros_like(data) 
for est in range(len(data)):
    gauss_fit_mat[est] = gauss_fit[est]
    gauss_pred_mat[est,:] = gauss_model.return_prediction(mu_x=gauss_fit[est][0], 
                                                          mu_y=gauss_fit[est][1], 
                                                          size=gauss_fit[est][2], 
                                                          beta=gauss_fit[est][3], 
                                                          baseline=gauss_fit[est][4],
                                                          hrf_1=gauss_fit[est][5],
                                                          hrf_2=gauss_fit[est][6])

# save across iteration
gauss_fit_mat_repeat[:,:,0] = gauss_fit_mat
gauss_pred_mat_repeat[:,:,0] = gauss_pred_mat
gauss_duration_repeat[0] = duration.total_seconds()

Grid fit:			2023-12-15 16:48:14.691287
End time:			2023-12-15 16:48:20.511661
Duration:			0:00:05.820374


In [12]:
# gaussian iterative fit
# ----------------------

# define parameters dict
gauss_iterative_dict = dict(rsq_threshold=rsq_iterative_th, 
                            verbose=iterative_verbose,
                            xtol=1e-4,
                            ftol=1e-4)

# define iterative bounds
gauss_bounds = [(-1.5*max_ecc_size, 1.5*max_ecc_size), # x
                (-1.5*max_ecc_size, 1.5*max_ecc_size), # y
                (0.001, 1.5*max_ecc_size), # prf size
                (0.001, 2.0), # prf amplitude
                (-1.0, 1.0), # bold baseline
                (0.001, 10.0), # hrf_1 bounds
                (0.001, 5.0)] # hrf_2 bounds  

# run iterative fit through repetitions
for repeat_num in np.arange(0,gauss_iterative_repeat+1,1):

    # if repeat_num = 0 => no bound
    # if repeat_num >= 1 => bounds
    if repeat_num == 1:
        gauss_iterative_dict['bounds'] = gauss_bounds
    if repeat_num >= 2:
        gauss_iterative_dict['starting_params'] = gauss_fitter.iterative_search_params

    start_time = datetime.datetime.now()
    gauss_fitter.iterative_fit(**gauss_iterative_dict)
    gauss_fit = gauss_fitter.iterative_search_params
    clear_output(wait=True)

    end_time = datetime.datetime.now()
    duration = end_time - start_time
    print("Iterative fit (repeat={}):\t{}\nEnd time:\t\t\t{}\nDuration:\t\t\t{}".format(
        repeat_num, start_time, end_time, duration))
    
    # arrange data
    gauss_fit_mat = np.zeros((data.shape[0],gauss_params_num))
    gauss_pred_mat = np.zeros_like(data) 
    for est in range(len(data)):
        gauss_fit_mat[est] = gauss_fit[est]
        gauss_pred_mat[est,:] = gauss_model.return_prediction(mu_x=gauss_fit[est][0], 
                                                              mu_y=gauss_fit[est][1], 
                                                              size=gauss_fit[est][2], 
                                                              beta=gauss_fit[est][3], 
                                                              baseline=gauss_fit[est][4],
                                                              hrf_1=gauss_fit[est][5],
                                                              hrf_2=gauss_fit[est][6]
                                                             )
        
    # save across iteration
    gauss_fit_mat_repeat[:,:,repeat_num+1] = gauss_fit_mat
    gauss_pred_mat_repeat[:,:,repeat_num+1] = gauss_pred_mat
    gauss_duration_repeat[repeat_num+1] = duration.total_seconds()

Iterative fit (repeat=15):	2023-12-15 16:49:18.489835
End time:			2023-12-15 16:49:20.453565
Duration:			0:00:01.963730


In [30]:
# save arrays
np.save(file='{}/{}_gauss_fit_mat_repeat.npy'.format(prf_fit_test_dir, subject),
        arr=gauss_fit_mat_repeat)
np.save(file='{}/{}_gauss_pred_mat_repeat.npy'.format(prf_fit_test_dir, subject),
        arr=gauss_pred_mat_repeat)
np.save(file='{}/{}_gauss_duration_repeat.npy'.format(prf_fit_test_dir, subject),
        arr=gauss_duration_repeat)

### CSS model

In [20]:
# define CSS model
css_model = CSS_Iso2DGaussianModel(stimulus=stimulus)

# grid fit CSS model
css_fitter = CSS_Iso2DGaussianFitter(data=data, 
                                     model=css_model, 
                                     n_jobs=n_jobs,
                                     use_previous_gaussian_fitter_hrf=False,
                                     previous_gaussian_fitter=gauss_fitter
                                    )

# fit parameters
css_params_num = 9
css_iterative_bound = True
css_iterative_repeat = 15

In [23]:
# CSS grid fit
# ------------

# define saving data matrices
css_fit_mat_repeat = np.zeros((data.shape[0], css_params_num, css_iterative_repeat+2))
css_pred_mat_repeat = np.zeros((data.shape[0], data.shape[1], css_iterative_repeat+2))
css_duration_repeat = np.zeros(css_iterative_repeat+2)

# defind grid values
exponent_css_grid = np.linspace(0, 4, css_grid_nr)

# run grid fit
start_time = datetime.datetime.now()
css_fitter.grid_fit(exponent_grid=exponent_css_grid,
                    verbose=grid_verbose,
                    n_batches=n_batches)

clear_output(wait=True)

end_time = datetime.datetime.now()
duration = end_time - start_time
print("Grid fit:\t\t\t{}\nEnd time:\t\t\t{}\nDuration:\t\t\t{}".format(
    start_time, end_time, duration))

# arrange data
css_fit = css_fitter.gridsearch_params
css_fit_mat = np.zeros((data.shape[0],css_params_num))
css_pred_mat = np.zeros_like(data) 
for est in range(len(data)):
    css_fit_mat[est] = css_fit[est]
    css_pred_mat[est,:] = css_model.return_prediction(mu_x=css_fit[est][0],
                                                      mu_y=css_fit[est][1], 
                                                      size=css_fit[est][2], 
                                                      beta=css_fit[est][3], 
                                                      baseline=css_fit[est][4],
                                                      n=css_fit[est][5],
                                                      hrf_1=css_fit[est][6],
                                                      hrf_2=css_fit[est][7])
# save across iteration
css_fit_mat_repeat[:,:,0] = css_fit_mat
css_pred_mat_repeat[:,:,0] = css_pred_mat
css_duration_repeat[0] = duration.total_seconds()

Grid fit:			2023-12-15 15:01:16.585985
End time:			2023-12-15 15:01:16.871760
Duration:			0:00:00.285775


In [24]:
# CSS iterative fit
# -----------------

# define parameters dict
css_iterative_dict = dict(rsq_threshold=rsq_iterative_th, 
                          verbose=iterative_verbose,
                          xtol=1e-4,
                          ftol=1e-4)

# define iterative bounds
css_bounds = [(-1.5*max_ecc_size, 1.5*max_ecc_size),  # x
              (-1.5*max_ecc_size, 1.5*max_ecc_size),  # y
              (0.001, 1.5*max_ecc_size),# prf size
              (0.001, 2.0), # prf amplitude
              (-1.0, 1.0), # bold baseline
              (0.001, 3.0), # CSS exponent
              (0.001, 10.0), # hrf_1 bounds
              (0.001, 5.0)] # hrf_2 bounds 

# run iterative fit through repetitions
for repeat_num in np.arange(0,css_iterative_repeat+1,1):

    # if repeat_num = 0 => no bound
    # if repeat_num >= 1 => bounds
    if repeat_num == 1:
        css_iterative_dict['bounds'] = css_bounds
    if repeat_num >= 2:
        css_iterative_dict['starting_params'] = css_fitter.iterative_search_params

    start_time = datetime.datetime.now()
    css_fitter.iterative_fit(**css_iterative_dict)
    css_fit = css_fitter.iterative_search_params
    clear_output(wait=True)

    end_time = datetime.datetime.now()
    duration = end_time - start_time
    print("Iterative fit (repeat={}):\t{}\nEnd time:\t\t\t{}\nDuration:\t\t\t{}".format(
        repeat_num, start_time, end_time, duration))
    
    # arrange data
    css_fit_mat = np.zeros((data.shape[0],css_params_num))
    css_pred_mat = np.zeros_like(data) 
    for est in range(len(data)):
        css_fit_mat[est] = css_fit[est]
        css_pred_mat[est,:] = css_model.return_prediction(mu_x=css_fit[est][0],
                                                          mu_y=css_fit[est][1], 
                                                          size=css_fit[est][2], 
                                                          beta=css_fit[est][3], 
                                                          baseline=css_fit[est][4],
                                                          n=css_fit[est][5],
                                                          hrf_1=css_fit[est][6],
                                                          hrf_2=css_fit[est][7])
        
    # save across iteration
    css_fit_mat_repeat[:,:,repeat_num+1] = css_fit_mat
    css_pred_mat_repeat[:,:,repeat_num+1] = css_pred_mat
    css_duration_repeat[repeat_num+1] = duration.total_seconds()

Iterative fit (repeat=15):	2023-12-15 15:02:21.411939
End time:			2023-12-15 15:02:23.403309
Duration:			0:00:01.991370


In [31]:
# save arrays
np.save(file='{}/{}_css_fit_mat_repeat.npy'.format(prf_fit_test_dir, subject),
        arr=css_fit_mat_repeat)
np.save(file='{}/{}_css_pred_mat_repeat.npy'.format(prf_fit_test_dir, subject),
        arr=css_pred_mat_repeat)
np.save(file='{}/{}_css_duration_repeat.npy'.format(prf_fit_test_dir, subject),
        arr=css_duration_repeat)